# Setup

Import a few common modules

In [1]:
#CODE for points = 1
#import sklearn, numpy, os

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
import numpy as np
import sklearn
import os

%load_ext tensorboard



# to make this notebook's output stable across runs
np.random.seed(42)


# Vanishing/Exploding Gradients Problem

In [2]:
def logit(z):
    return 1 / (1 + np.exp(-z))

In [3]:
z = np.linspace(-5, 5, 200)

## Xavier and He Initialization

In [4]:
#CODE for points = 1
# set activation to relu and kernel initializer to he_normal

keras.layers.Dense(10, activation= "relu", kernel_initializer= "he_normal")

### Leaky ReLU

In [5]:
#CODE for points = 1

# remember α is the hyperparameter that defines how much the function “leaks”
# set the value of alpha, use the value typically set

def leaky_relu(z, alpha= .01):
    return np.maximum(alpha*z, z)

Let's train a neural network on Fashion MNIST using the Leaky ReLU:

In [6]:
#CODE for points = 1
#load MNIST dataset from keras

import keras


(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data(path="mnist.npz")
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [7]:
#CODE for points = 1

# initialize kernel_initializer to "he_normal" and activation function to softmax

tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer= "he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer= "he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation= "softmax")
])

2021-11-11 08:37:25.099623: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
from tensorflow.keras import optimizers

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [9]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

2021-11-11 08:37:28.805856: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
1719/1719 [==============================] - 6s 3ms/step - loss: 1.5275 - accuracy: 0.5970 - val_loss: 0.9444 - val_accuracy: 0.7980
Epoch 2/5
1719/1719 [==============================] - 8s 4ms/step - loss: 0.7465 - accuracy: 0.8287 - val_loss: 0.5868 - val_accuracy: 0.8596
Epoch 3/5
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5412 - accuracy: 0.8624 - val_loss: 0.4685 - val_accuracy: 0.8834
Epoch 4/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4591 - accuracy: 0.8771 - val_loss: 0.4104 - val_accuracy: 0.8940
Epoch 5/5
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4142 - accuracy: 0.8869 - val_loss: 0.3758 - val_accuracy: 0.9006


Now look at what happens if we try to use the ReLU activation function instead:

In [10]:
np.random.seed(42)
tf.random.set_seed(42)

In [11]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))

model.add(keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"))

for layer in range(99):
    model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"))

model.add(keras.layers.Dense(10, activation="softmax"))

In [12]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [13]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 33s 15ms/step - loss: 1.9449 - accuracy: 0.2574 - val_loss: 1.5999 - val_accuracy: 0.3436
Epoch 2/5
1719/1719 [==============================] - 21s 12ms/step - loss: 1.3867 - accuracy: 0.4122 - val_loss: 1.1243 - val_accuracy: 0.5034
Epoch 3/5
1719/1719 [==============================] - 20s 12ms/step - loss: 1.3844 - accuracy: 0.4257 - val_loss: 1.0929 - val_accuracy: 0.5238
Epoch 4/5
1719/1719 [==============================] - 21s 12ms/step - loss: 0.9934 - accuracy: 0.6115 - val_loss: 0.7235 - val_accuracy: 0.7388
Epoch 5/5
1719/1719 [==============================] - 23s 14ms/step - loss: 0.8315 - accuracy: 0.7089 - val_loss: 0.7754 - val_accuracy: 0.7484


Not great at all, we suffered from the vanishing/exploding gradients problem.

# Batch Normalization

In [14]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_105 (Dense)            (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_106 (Dense)            (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_107 (Dense)            (None, 10)               

In [16]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [17]:
#bn1.updates #deprecated

In [18]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 9s 5ms/step - loss: 0.8619 - accuracy: 0.7334 - val_loss: 0.4687 - val_accuracy: 0.8716
Epoch 2/5
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4346 - accuracy: 0.8733 - val_loss: 0.3494 - val_accuracy: 0.9038
Epoch 3/5
 617/1719 [=========>....................] - ETA: 4s - loss: 0.3767 - accuracy: 0.8900

Sometimes applying BN before the activation function works better (there's a debate on this topic). Moreover, the layer before a `BatchNormalization` layer does not need to have bias terms, since the `BatchNormalization` layer some as well, it would be a waste of parameters, so you can set `use_bias=False` when creating those layers:

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

# Faster Optimizers

## Momentum optimization

In [ ]:
#CODE for points = 1
# initialize lr and momentum to typical values

optimizer = optimizers.SGD(learning_rate= .01, momentum= .9)

## Nesterov Accelerated Gradient

In [ ]:
#CODE for points = 1
# initialize lr and momentum to typical values. Set nesterov so that it is used

optimizer = optimizers.SGD(learning_rate=.01, momentum= .9, nesterov=.1)

# Avoiding Overfitting Through Regularization

## $\ell_1$ and $\ell_2$ regularization

In [ ]:
#CODE for points = 0.5

from tensorflow.keras import regularizers

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=tf.keras.regularizers.l2(l2=0.01))

In [ ]:
#CODE for points = 1

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/
# nadam optimizer

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=tf.keras.regularizers.l2(l2=0.01))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer= "nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

In [ ]:
#CODE for points = .5

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=tf.keras.regularizers.l2(l2=0.01))

In [ ]:
#CODE for points = 1

# activation function initialized as softmax
# nadam optimizer


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation= "softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))
